In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of the dataset in characters: ", len(text))

length of the dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}
encode = lambda s: [stoi[ch]for ch in s]
decode = lambda l: [itos[i] for i in l] # decoder: take a list of integers, output a string

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
['h', 'e', 'l', 'l', 'o']


In [7]:
# Let's now encode the entire dataset into a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x11661fb10>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15,

In [8]:
# Let's now split the data into training and validation sets
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:] # I think there is a more proper way to do this, but this is fine for now

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(context, "->", target)

tensor([18]) -> tensor(47)
tensor([18, 47]) -> tensor(56)
tensor([18, 47, 56]) -> tensor(57)
tensor([18, 47, 56, 57]) -> tensor(58)
tensor([18, 47, 56, 57, 58]) -> tensor(1)
tensor([18, 47, 56, 57, 58,  1]) -> tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) -> tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> tensor(58)


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data         # select the data source
    ix = torch.randint(len(data)-block_size, (batch_size,))     # sample batch_size starting indices/charecter
    x = torch.stack([data[i   : i+block_size]   for i in ix])   # stack the charecters to make input - (batch_size, block_size)
    y = torch.stack([data[i+1 : i+1+block_size] for i in ix])   # stack the charecters to make target, shifted by 1 - (batch_size, block_size)

    return x, y

xb, yb = get_batch('train')
print("inputs")
print(xb.shape)
print(xb)
print("outputs")
print(yb.shape)
print(yb)

print('-----')

# (batch_size, block_size, block_size)
for b in range (batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f'{context} -> {target}')

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
tensor([24]) -> 43
tensor([24, 43]) -> 58
tensor([24, 43, 58]) -> 5
tensor([24, 43, 58,  5]) -> 57
tensor([24, 43, 58,  5, 57]) -> 1
tensor([24, 43, 58,  5, 57,  1]) -> 46
tensor([24, 43, 58,  5, 57,  1, 46]) -> 43
tensor([24, 43, 58,  5, 57,  1, 46, 43]) -> 39
tensor([44]) -> 53
tensor([44, 53]) -> 56
tensor([44, 53, 56]) -> 1
tensor([44, 53, 56,  1]) -> 58
tensor([44, 53, 56,  1, 58]) -> 46
tensor([44, 53, 56,  1, 58, 46]) -> 39
tensor([44, 53, 56,  1, 58, 46, 39]) -> 58
tensor([44, 53, 56,  1, 58, 46, 39, 58]) -> 1
tensor([52]) -> 58
tensor([52, 58]) -> 1
tensor([52, 58,  1]) -> 58
tensor([52,

In [12]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__() # to initialize the nn.Module base class
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # initialize a matrix of shape (vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C), (batch_size, block_size, vocab_size)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) # since we don't want to deal with (B,C,T) tensors expected for cross_entropy. we instead flatten the first two dimensions
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            # 

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idk is (B,T) array of indeces in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # get the predictions
            logits = logits[:, -1, :] # focus only on the last time step - become (B,C)
            probs = F.softmax(logits, dim=-1) # apply softmax to get the probabilities - (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) # sample from the distribution - (B,1)
            idx = torch.cat((idx, idx_next), dim=1) # append sample index to the running sequence - (B, T+1)
        return idx
    
    
# In the current example there is a lot of wasted compute in generate as we are computing logits for all tokens, instead of just the last one.
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print("".join(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist())))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [14]:
# create a Pytorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32

for steps in range(100): # increase the number of steps for better results
    xb, yb = get_batch('train') # sample a batch of data

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward() # gets the gradients
    optimizer.step() # updates the parameters

print(loss.item())

4.657352447509766


In [16]:
print(''.join(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist())))


oTo.JUZ!!zqe!
xBP qbs$Gy'AcOmrLwwt
p$x;Seh-onQbfM?OjKbn'NwUAW -Np3fkz$FVwAUEa-wfgC -wQo-R!v -Mj?,SPiTyZ;o-opr$mOiPJEYD-CfigkzD3p3?zvS;ADz;.y?o,ivCuC'zqHxcVT cHA
rT'Fd,SBMZyOslg!NXeF$sBe,juUzLq?w-wzP-h
ERjjxlgJzPbHxf$ q,q,KCDCU fqBOQT
SV&CW:xSVwZv'DG'NSPypDhKStKzC -$hslxIVzoivnp ,ethA:NCCGoi
tN!ljjP3fwJMwNelgUzzPGJlgihJ!d?q.d
pSPYgCuCJrIFtb
jQXg
pA.P LP,SPJi
DBcuBM:CixjJ$Jzkq,OLf3KLQLMGph$O 3DfiPHnXKuHMlyjxEiyZib3FaHV-oJa!zoc'XSP :CKGUhd?lgCOF$;;DTHZMlvvcmZAm;:iv'MMgO&Ywbc;BLCUd&vZINLIzkuTGZa
D.?


# The Mathematical Trick in Self-Attention

In [17]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
print('a=')
print(a)
print('--')
a = a / torch.sum(a, 1, keepdim=True) # the column of sums is duplicated for each columns
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [18]:
# consider the following toy example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape
print(x)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [19]:
# In these example we simply want to compute the mean of all the previous tokens at each time step t
# Later, instead of computing the mean, we will compute the attention over the previous tokens

# Version 1: compute the mean using for loop
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # bow = bag of words, stands for mean of words/chars
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # t,C - the previous tokens - it increases by one at each step
        xbow[b,t] = torch.mean(xprev, dim=0) # C=2 - mean over the time dimension
        # print(xprev)
        # print(xbow[b,t])
        # print('---')
        
    # break
# print(xbow.shape)
# print(xbow)


In [20]:
# Version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones((T,T)))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # we weighted/averaged it by the previous tokens - later we will update using attention of the previous tokens
torch.allclose(xbow, xbow2)

True

In [21]:
# Version: use softmax
# Instead of starting with a matrix of ones, we start with a matrix of zeros and then fill the lower triangular part with -inf
# We use the softmax function to "normalize" the weights
# Hence, wei value can now adjust to the importance of the previous tokens soon. 
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf')) 
print(wei)
wei = F.softmax(wei, dim=-1)
print(wei)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [ ]:
# Version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32          # batch, time, channels - channels in practice is the embedding size - set of numbers/weights that represent the token/charecter
x = torch.randn(B,T,C)  # B,T,C - word embedding - word in numbers

# let's see a single head perform a self-attention
head_size = 16 # headsize is the size of the output of the self-attention head - it is the number of features we want to learn

query = nn.Linear(C, head_size, bias=False) # learnable weights per token/character to soon be plucked - value to compare to key
key   = nn.Linear(C, head_size, bias=False) # learnable weights per token/character to soon be plucked - value to compare to query
value = nn.Linear(C, head_size, bias=False) # learnable weights per token/character to soon be plucked - how much to update

q = query(x)    # (B,T,H) - plucked token base on what is the current Time step - component of the attention matrix - "this is what I need" - because token values of header size
k = key(x)      # (B,T,H) - plucked token base on what is the current Time step - component of the attention matrix - "this is what I am" - because token values of header size will be compared to q
                # differs from q because  of the shape being transposed which results to (B,H,T). Hence, tril kills the future tokens

wei = q @ k.transpose(-2, -1)   # (B, T, 16) @ (B, 16, T) ---> (B, T, T) - attention / importance matrix - how much I should pay attention to the previous tokens - count
                                # combination of q and k - where both give n>0 and same sign - the more similar the more attention

tril = torch.tril(torch.ones((T,T)))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1) # only able to see current and previous tokens attention / importance matrix - probabilities

v = value(x)    # (B,T,H) # plucked token base on what is the current Time step - value to update
out = wei @ v   # (B,T,T) @ (B,T,H) ---> (B,T,H) - updated base on the attention/importance of the previous tokens

out.shape

torch.Size([4, 8, 16])